#                                                       PIPELINE

##  Pedir Archivos De Github

Codigo automatizado con Cloud Scheduler para que se active cada cierto tiempo. Agarra los Datasets de Github en crudo y los tira en Cloud Store.

El token para autenticar el github fue sacado y reemplazado por "x" por seguridad.

In [ ]:
import requests
from google.cloud import storage

def descargar_y_cargar_archivos_gcs(request):
    # Token de autenticación para la API de GitHub
    github_token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

    # URL del repositorio y ruta donde se encuentran los archivos
    repo_url = "https://api.github.com/repos/darksider10/PF/contents/Datasets"

    # Encabezados para la solicitud con el token de autenticación
    headers = {
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3+json"
    }

    # Realizar solicitud GET a la API de GitHub
    response = requests.get(repo_url, headers=headers)
    data = response.json()

    # Crear una instancia de cliente de Google Cloud Storage
    client = storage.Client()

    # Obtener un objeto de cubo (bucket) de almacenamiento
    bucket = client.get_bucket('dastorage')

    # Descargar archivos y cargarlos a Google Cloud Storage
    for file_data in data:
        if file_data["type"] == "file" and (file_data["name"].endswith('.dbf') or file_data["name"].endswith('.csv')):
            file_url = file_data["download_url"]
            file_name = file_data["name"]
            file_content = requests.get(file_url, headers=headers).content

#Crear un objeto Blob en el cubo de almacenamiento
            blob = bucket.blob(file_name)

#Cargar el contenido del archivo al objeto Blob
            blob.upload_from_string(file_content)

    return "Archivos cargados exitosamente en Google Cloud Storage."

Web Scrapping con archivos de Taxis amarillos del año 2022, se agarran los 12 meses del año por separados y se carga en Cloud Storage con esta Cloud Function.

In [ ]:
import requests
from bs4 import BeautifulSoup
from google.cloud import storage

def download_and_upload_parquet_files(request):
    # URL del sitio web a realizar el web scraping
    website = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    response = requests.get(website)
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrar enlaces que apunten a archivos .parquet
    links = soup.find_all("a", href=lambda href: href and "yellow_tripdata_2022" in href and ".parquet")

    # Inicializar el cliente de Google Cloud Storage
    storage_client = storage.Client()
    bucket = storage_client.bucket('parquetsws')  # Reemplaza 'tu_bucket' con el nombre de tu bucket

    for link in links:
        # Descargar el archivo .parquet
        file_url = link["href"]
        file_name = file_url.split("/")[-1]  # Obtener el nombre del archivo de la URL
        response = requests.get(file_url)
        if response.status_code == 200:
            # Subir el archivo .parquet a Google Cloud Storage
            blob = bucket.blob(file_name)
            blob.upload_from_string(response.content)
            print(f"Archivo {file_name} subido a Google Cloud Storage.")
        else:
            print(f"No se pudo descargar el archivo {file_url}.")

    return "Proceso completado."

#   Carga Parquets a BigQuery

Unifica todos los parquets en uno solo y modifica el codigo, como son demasiados registros solo dejo 10.000 por cada mes y lo guardo.

Esta Cloud Function es un Trigger y se activa cuando entran archivos .parquet al Bucket 

In [ ]:
from google.cloud import storage, bigquery
import pyarrow.parquet as pq
import io
import pandas as pd

def create_dataset(project_id, dataset_id):
    bigquery_client = bigquery.Client(project=project_id)
    
    dataset_ref = bigquery_client.dataset(dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    
    try:
        bigquery_client.create_dataset(dataset)
        print(f"Dataset {dataset_id} creado exitosamente en BigQuery.")
    except Exception as e:
        print(f"Error al crear el dataset: {e}")

def process_file(data, context):
    # Obtiene la información del archivo creado
    bucket_name = data['bucket']
    file_name = data['name']

    # Configura el cliente de Google Cloud Storage y BigQuery
    storage_client = storage.Client()
    bigquery_client = bigquery.Client()

    # Lee el contenido del archivo desde Cloud Storage
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    file_content = blob.download_as_string()

    # Utiliza io.BytesIO para crear un objeto de archivo en memoria
    file_content_as_file = io.BytesIO(file_content)

    # Define el nombre de la tabla en BigQuery
    table_id = 'yellow_tripdata_2022'

    if file_name.endswith('.parquet'):
        # Lee el archivo Parquet usando pyarrow
        table = pq.read_table(file_content_as_file)

        # Convierte la tabla Parquet a un DataFrame de pandas
        df_parquet = table.to_pandas()

        df_parquet["PickupZone"] = df_parquet["PULocationID"].apply(get_borough)

        df_parquet = df_parquet.dropna()

        # Selecciona solo las columnas requeridas
        df_parquet = df_parquet.sample(n=10000, random_state=42)[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 
                                        'passenger_count', 'trip_distance', 
                                        'PULocationID', 'DOLocationID', 
                                        'total_amount','PickupZone']]
        

        # Crea un archivo temporal CSV para cargar en BigQuery
        csv_content_as_file = io.StringIO()
        df_parquet.to_csv(csv_content_as_file, index=False)

        # Mueve el cursor al inicio del archivo CSV
        csv_content_as_file.seek(0)

        # Configura el job de carga en BigQuery
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV,
            autodetect=True,
        )

        # Carga el contenido del archivo CSV en BigQuery
        table_ref = bigquery_client.dataset('ETL').table(table_id)
        load_job = bigquery_client.load_table_from_file(csv_content_as_file, table_ref, job_config=job_config)
        load_job.result()  # Espera a que termine la carga

        print(f'Todos los archivos Parquet han sido cargados en BigQuery con el nombre de tabla {table_id}.')

def get_borough(location_id):
            # Agrupamos los PULocationID y DOLocationID de acuerdo a su Borough correspondiente
    zones = {   #Manhattan
                "Manhattan" : [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 103, 103, 107, 113, 114, 116, 120, 125,
                            127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211,
                            224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263],

                #Brooklyn
                "Brooklyn" : [11, 14, 17, 21, 22, 25, 26, 29, 33, 34, 35, 36, 37, 39, 40, 49, 52, 54, 55, 61, 62, 63, 65, 66, 67, 71, 72, 76, 77, 80,
                            85, 89, 91, 97, 106, 108, 111, 112, 123, 133, 149, 150, 154, 155, 165, 177, 178, 181, 188, 189, 190, 195, 210, 217, 222,
                            225, 227, 228, 255, 256, 257],

                #Queens
                "Queens" : [2, 7, 8, 9, 10, 15, 16, 19, 27, 28, 30, 38, 53, 56, 56, 64, 70, 73, 82, 83, 86, 92, 93, 95, 96, 98, 101, 102, 117, 121,
                        122, 124, 129, 130, 131, 132, 134, 135, 138, 139, 145, 146, 157, 160, 171, 173, 175, 179, 180, 191, 192, 193, 196, 197,
                        198, 201, 203, 205, 207, 215, 216, 218, 219, 223, 226, 252, 253, 258, 260],

                #Bronx
                "Bronx" : [3, 18, 20, 31, 32, 46, 47, 51, 58, 59, 60, 69, 78, 81, 94, 119, 126, 136, 147, 159, 167, 168, 169, 174, 182, 183, 184,
                        185, 199, 200, 208, 212, 213, 220, 235, 240, 241, 242, 247, 248, 250, 254, 259],
                #State Island
                "State Island" : [5, 6, 23, 44, 84, 99, 109, 110, 115, 118, 156, 172, 176, 187, 204, 206, 214, 221, 245, 251],

                #EWR
               "EWR" : [1]}

    for borough, ids in zones.items():
        if location_id in ids:
            return borough
    return "Unknow"

#   ETL de Cloud Storage a BigQuery de Archivos .csv Crudos

Esta funcion es un Trigger que se activa cuando entran los archivos .csv al Bucket.

In [ ]:
from google.cloud.exceptions import NotFound
from google.cloud import storage, bigquery
import pandas as pd
import io
import numpy as np


def process_file(data, context):
    # Obtiene la información del archivo creado
    bucket_name = data['bucket']
    file_name = data['name']

    # Configura el cliente de Google Cloud Storage y BigQuery
    storage_client = storage.Client()
    bigquery_client = bigquery.Client()

    # Lee el contenido del archivo desde Cloud Storage
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    file_content = blob.download_as_text()
    
    # Si el archivo es 'Alternative Fuel Vehicles US.csv', aplicamos transformaciones
    if 'Alternative_Fuel_Vehicles_US.csv' in file_name:
        # Realiza transformaciones en el archivo y guarda el resultado en file_content_as_file
        
        # Transformar el contenido del archivo en un DataFrame de pandas
        df = pd.read_csv(io.StringIO(file_content))

        #-------------------------------------------------------------------------------------
        # Eliminar columnas innecesarias del DataFrame
        df.drop(columns=['Heavy-Duty Power System', 'Notes', 'Drivetrain', 'Number of Passengers',
        'PHEV Total Range', 'Engine Cylinder Count', 'Transmission Make', 'Transmission Type', 
        'All-Electric Range', 'Alternative Fuel Economy City', 'Alternative Fuel Economy Highway', 'Alternative Fuel Economy Combined',
        'Conventional Fuel Economy City', 'Conventional Fuel Economy Highway', 'Conventional Fuel Economy Combined'], axis=1, inplace=True)

        # Definir las categorías a eliminar
        categories_to_remove = [
            'Refuse', 'School Bus', 'Street Sweeper', 'Tractor',
            'TractorVocational/Cab Chassis', 'Transit Bus',
            'Vocational/Cab Chassis', 'Vocational/Cab ChassisTractor',
            'Passenger Van/Shuttle Bus', 'Van', 'Pickup', 'Step Van', 'Vocational/Cab ChassisVan'
        ]

        # Filtrar el DataFrame para eliminar las categorías especificadas
        df = df[~df['Category'].isin(categories_to_remove)]

        # Normalizar los valores de la columna 'Engine Size' para que todos tengan el formato 'X kW'
        df['Engine Size'] = df['Engine Size'].str.replace(r'(\d+)\s*[kK][wW].*', r'\1 kW', regex=True)

        # Reemplazar abreviaturas en la columna 'Engine Type' por su nombre completo
        df['Engine Type'] = df['Engine Type'].replace({
            'SI': 'Spark Ignition',
            'e-motor': 'Electric Motor',
            'CI': 'Compression Ignition',
            'FC': 'Fuel Cell'
        })

        # Eliminar filas con valores faltantes en las columnas 'Engine Type' y 'Engine Size'
        df = df.dropna(subset=['Engine Type', 'Engine Size'])

        # Eliminar filas duplicadas
        df = df.drop_duplicates()

        # Renombrar las columnas del DataFrame
        df.columns = [
            'Categoría', 'Modelo', 'AñoModelo', 'Fabricante', 
            'Combustible', 'TipoMotor', 'TamañoMotor'
        ]

        df["ID"] = df.index

        # Convertir el DataFrame de pandas transformado de nuevo a un archivo CSV en memoria
        transformed_file = io.StringIO()
        df.to_csv(transformed_file, index=False)
        transformed_file.seek(0)
        
        # Usaremos 'transformed_file' en lugar de 'file_content_as_file' para la carga en BigQuery
        file_content_as_file = transformed_file
        # Define el nombre de la tabla en BigQuery usando el nombre del archivo sin extensión como tabla
        table_id = file_name.split('.')[0]
    
    elif 'Electric_and_Alternative_Fuel_Charging_Stations.csv' in file_name:
       
        df_ny = pd.read_csv(io.StringIO(file_content))

        #-------------------------------------------------------------------------------
        # Filtrar las filas donde la columna 'State' es igual a 'NY'
        df_ny = df_ny[df_ny['State'] == 'NY']

        # Lista de columnas a eliminar
        columns_to_drop = [
            "Intersection Directions", "Plus4", "Expected Date", "Cards Accepted",
            "BD Blends", "NG Fill Type Code", "NG PSI", "EV Other Info",
            "Federal Agency ID", "Federal Agency Name", "Federal Agency Code",
            "Geocode Status", "Date Last Confirmed", "Owner Type Code", "Facility Type",
            "CNG Dispenser Num", "CNG On-Site Renewable Source",
            "CNG Total Compression Capacity", "CNG Storage Capacity",
            "LNG On-Site Renewable Source", "E85 Other Ethanol Blends",
            "LPG Nozzle Types", "Hydrogen Pressures", "Hydrogen Standards",
            "CNG Fill Type Code", "CNG PSI", "CNG Vehicle Class",
            "LNG Vehicle Class", "EV On-Site Renewable Source",
            "Intersection Directions (French)", "Access Days Time (French)",
            "BD Blends (French)", "Groups With Access Code (French)", "Hydrogen Status Link", "LPG Primary",
            "E85 Blender Pump", "Hydrogen Is Retail", "EV Connector Types", "ZIP", "EV Level2 EVSE Num", "Open Date","EV Pricing",
            "EV DC Fast Count", "Restricted Access", "Access Days Time", "Access Detail Code", "Updated At", "Status Code", "Access Code"
        ]

        # Eliminar las columnas del dataframe
        df_ny.drop(columns=columns_to_drop, inplace=True, errors='ignore')

        # Aplicar una función para simplificar los valores de "Groups With Access Code" a "Public" o "Private"
        df_ny['Groups With Access Code'] = df_ny['Groups With Access Code'].apply(lambda x: "Public" if "Public" in x else ("Private" if "Private" in x else x))

        # Eliminar columnas  del DataFrame
        columns_to_drop = ['Station Phone', 'EV Network', 'EV Network Web', 'NG Vehicle Class', 'Country', 'EV Pricing (French)', 'EV Level1 EVSE Num' ]
        df_ny.drop(columns=columns_to_drop, inplace=True)

        # Traducción y Renombramiento de columnas al español
        columnas_en_espanol = {
            'Fuel Type Code': 'CodigoCombustible',
            'Station Name': 'NombreEstacion',
            'Street Address': 'Dirección',
            'City': 'Ciudad',
            'State': 'Estado',
            'Groups With Access Code': 'AccesoPublico',
            'Latitude': 'Latitud',
            'Longitude': 'Longitud',
            'ID': 'IDEstacion',
        }
        df_ny.rename(columns=columnas_en_espanol, inplace=True)

        # ajuste de datos
        df_ny['AccesoPublico'] = df_ny['AccesoPublico'].replace({'Public': True, 'Private': False})

        df_ny["ID"] = df_ny.index

        transformed_file = io.StringIO()
        df_ny.to_csv(transformed_file, index=False)
        transformed_file.seek(0)
        
        file_content_as_file = transformed_file
        table_id = file_name.split('.')[0]

    elif 'Light_Duty_Vehicles.csv' in file_name:

        df = pd.read_csv(io.StringIO(file_content))

        #------------------------------------------------------------------------------
        # Eliminar columnas que no vamos a utilizar porque no necesitamos la información o porque la misma se ennuentra disponible de forma más detallada en otro dataset.

        df.drop(columns=[ 'Notes', 'Drivetrain', 'PHEV Total Range', 'Engine Cylinder Count','Transmission Type', 'Engine Description','Fuel Configuration ID', 
        'Manufacturer URL', 'Fuel Code', 'PHEV Type', 'Alternative Fuel Economy City', 'Alternative Fuel Economy Highway','Alternative Fuel Economy Combined', 
        'Conventional Fuel Economy City', 'Conventional Fuel Economy Highway', 'Conventional Fuel Economy Combined', 'Engine Type', 'Engine Size', 
        'Fuel Configuration Name', 'Electric-Only Range'], axis=1, inplace=True)

        # Filtramos el dataset para incluir solo "Sedans" y "SUVs"
        df = df[df['Category'].isin(['Sedan', 'SUV'])]

        # Filtrar el DataFrame para incluir solo registros desde 2017 hasta 2022
        df = df[(df['Model Year'] >= 2017) & (df['Model Year'] <= 2022)]

        # Cambiar los nombres de las columnas al español
        df.columns = [
            'IDVehículo', 'IDCombustible', 'IDFabricante', 
            'IDCategoría', 'Modelo', 'AñoModelo', 
            'Fabricante', 'Categoría', 'Combustible'
        ]

        
        transformed_file = io.StringIO()
        df.to_csv(transformed_file, index=False)
        transformed_file.seek(0)
        
        file_content_as_file = transformed_file
        table_id = file_name.split('.')[0]

    elif 'Vehicle_Fuel_Economy_Data.csv' in file_name:

        df = pd.read_csv(io.StringIO(file_content))

        #------------------------------------------------------------------------------
        # Las columnas que vamos a dejar en el dataset por su importancia para nuestro objetivo
        columns_to_keep_corrected = [
            'Year', 'Manufacturer', 'Model', 'VClass', 'fuelType', 'fuelType1', 
            'city08', 'highway08', 'comb08', 
            'cityA08', 'highwayA08', 'combA08', 
            'co2', 'co2TailpipeGpm', 
            'phevBlended', 
            'range',
            'fuelCost08', 
            'fuelCostA08', 
        ]
        # Filtramos el dataframe para mantener solo las columnas seleccionadas
        df = df[columns_to_keep_corrected]


        # Eliminamos duplicados estrictos (donde todas las columnas relevantes son idénticas)
        df = df.drop_duplicates()


        # Filtrar el dataframe para incluir solo vehículos desde el año 2017 en adelante
        df = df[df['Year'] >= 2017]

        #  Cambiar los nombres de las columnas al español
        df.rename(columns={
            'Year': 'Año',
            'Manufacturer': 'Fabricante',
            'Model': 'Modelo',
            'VClass': 'ClaseVehículo',
            'fuelType': 'TipoCombustible',
            'fuelType1': 'TipoCombustible1',
            'city08': 'Ciudad08',
            'highway08': 'Carretera08',
            'comb08': 'Combinado08',
            'cityA08': 'CiudadA08',
            'highwayA08': 'CarreteraA08',
            'combA08': 'CombinadoA08',
            'co2': 'CO2',
            'co2TailpipeGpm': 'CO2TuboEscapeGpm',
            'phevBlended': 'HibridoEnchufable',
            'range': 'Rango',
            'fuelCost08': 'CostoCombustible08',
            'fuelCostA08': 'CostoCombustibleA08'
        }, inplace=True)

        # Eliminar 'TipoCombustible' y renombrar 'TipoCombustible1' a 'TipoCombustible'
        df.drop(columns=['TipoCombustible'], inplace=True)
        df.rename(columns={'TipoCombustible1': 'TipoCombustible'}, inplace=True)

        # Eliminar 'CO2TuboEscapeGpm' y renombrar 'CO2' a 'CO2(g/pm)'
        df.drop(columns=['CO2TuboEscapeGpm'], inplace=True)
        df.rename(columns={'CO2': 'CO2(g/pm)'}, inplace=True)

        # Renombrar 'Rango' a 'RangoAutonomia'
        df.rename(columns={'Rango': 'RangoAutonomia'}, inplace=True)

        # Eliminar 'CostoCombustibleA08' y renombrar 'CostoCombustible08' a 'CostoCombustible'
        df.drop(columns=['CostoCombustibleA08'], inplace=True)
        df.rename(columns={'CostoCombustible08': 'CostoCombustible'}, inplace=True)

        # Eliminar columnas específicas de economía de combustible en ciudad y carretera
        df.drop(columns=['Ciudad08', 'Carretera08', 'CiudadA08', 'CarreteraA08'], inplace=True)

        # Renombrar columnas de eficiencia de combustible
        df.rename(columns={'Combinado08': 'EficienciaConv', 'CombinadoA08': 'EficienciaAlt'}, inplace=True)
        df = df.dropna()

        df["ID"] = df.index

        df["Eficiencia"] = np.where(df["EficienciaAlt"] != 0, ((300 * df["EficienciaConv"]) / (300 * df["EficienciaAlt"])) * 100, np.nan)

        """Eficiencia=(Km recorridos por vehıˊculo convencional×Consumo de combustible total de vehıˊculos 
        convencionalesKm recorridos por vehıˊculo eleˊctrico×Consumo energeˊtico total de vehıˊculos eleˊctricos​)×100"""

        transformed_file = io.StringIO()
        df.to_csv(transformed_file, index=False)
        transformed_file.seek(0)
        
        file_content_as_file = transformed_file
        table_id = file_name.split('.')[0]
        
    elif 'Batteries.csv' in file_name:

        df = pd.read_csv(io.StringIO(file_content))
        
        #-----------------------------------------------------------------------------
        df["ID"] = df.index
        
        transformed_file = io.StringIO()
        df.to_csv(transformed_file, index=False)
        transformed_file.seek(0)
        
        file_content_as_file = transformed_file
        table_id = file_name.split('.')[0]

    elif 'ElectricCarData_Clean.csv' in file_name:

        electric_cardata = pd.read_csv(io.StringIO(file_content))
        
        #-----------------------------------------------------------------------------
        electric_cardata["ID"] = electric_cardata.index
        
        transformed_file = io.StringIO()
        electric_cardata.to_csv(transformed_file, index=False)
        transformed_file.seek(0)
        
        file_content_as_file = transformed_file
        table_id = file_name.split('.')[0]
        

    else:
        # Si el archivo no es el esperado, pasa el contenido original a BigQuery
        file_content_as_file = io.StringIO(file_content)
        table_id = file_name.split('.')[0]
    
    # Configura el job de carga en BigQuery
    dataset_id = 'ETL'
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )

    # Carga el contenido del archivo en BigQuery
    dataset_ref = bigquery_client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    try:
        # Verifica si la tabla ya existe
        table = bigquery_client.get_table(table_ref)
        print(f'Tabla {table_id} ya existe en BigQuery. Eliminando...')
        # Si la tabla ya existe, elimina completamente la tabla
        bigquery_client.delete_table(table_ref)
        print(f'Tabla {table_id} eliminada de BigQuery.')
    except NotFound:
        # Si la tabla no existe, imprime un mensaje
        print(f'Tabla {table_id} no encontrada en BigQuery. Creando...')

    # Crea la tabla en BigQuery
    table = bigquery.Table(table_ref)
    table = bigquery_client.create_table(table)

    # Carga el contenido del archivo en BigQuery
    load_job = bigquery_client.load_table_from_file(file_content_as_file, table_ref, job_config=job_config)
    load_job.result()  # Espera a que termine la carga

    print(f'Archivo {file_name} cargado en BigQuery con el nombre de tabla {table_id}.')

#   Solucion De Problemas con los Parquets

Ya que la funcion para pasar los parquets los va sumando esta funcion borra la tabla.

In [ ]:
import os
from google.cloud import bigquery
from google.cloud import storage

def eliminar_tabla(event, context):
    # Obtener el nombre del archivo cargado
    file_name = event['name']

    # Verificar si el nombre del archivo coincide con el patrón esperado
    if 'yellow_tripdata_2022-01.parquet' in file_name:
        # Eliminar la tabla en BigQuery
        proyecto = "swift-climate-415514"
        dataset = "ETL"
        tabla_a_eliminar = f"{proyecto}.{dataset}.yellow_tripdata_2022"
        eliminar_tabla_bigquery(tabla_a_eliminar)
        print(f"La tabla {tabla_a_eliminar} ha sido eliminada.")

def eliminar_tabla_bigquery(tabla):
    # Inicializar el cliente de BigQuery
    bq_client = bigquery.Client()

    # Eliminar la tabla
    bq_client.delete_table(tabla, not_found_ok=True)

    # Imprimir mensaje de confirmación
    print(f"La tabla {tabla} ha sido eliminada.")



#   MACHINE LEARNING

El modelo fue entrenado en local y el Deploy se hizo con Streamlit conectando con Github donde se encontraba el .PKL que es el modelo ya entrenado.

#   Entrenamiento del modelo

Este modelo se entrena con ElasticNet que es un método de regularización utilizado en el aprendizaje automático y la estadística, especialmente en problemas de regresión.

In [ ]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

def get_borough(location_id):
            # Agrupamos los PULocationID y DOLocationID de acuerdo a su Borough correspondiente
    zones = {   #Manhattan
                1 : [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 103, 103, 107, 113, 114, 116, 120, 125,
                            127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211,
                            224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263],

                #Brooklyn
                2 : [11, 14, 17, 21, 22, 25, 26, 29, 33, 34, 35, 36, 37, 39, 40, 49, 52, 54, 55, 61, 62, 63, 65, 66, 67, 71, 72, 76, 77, 80,
                            85, 89, 91, 97, 106, 108, 111, 112, 123, 133, 149, 150, 154, 155, 165, 177, 178, 181, 188, 189, 190, 195, 210, 217, 222,
                            225, 227, 228, 255, 256, 257],

                #Queens
                3 : [2, 7, 8, 9, 10, 15, 16, 19, 27, 28, 30, 38, 53, 56, 56, 64, 70, 73, 82, 83, 86, 92, 93, 95, 96, 98, 101, 102, 117, 121,
                        122, 124, 129, 130, 131, 132, 134, 135, 138, 139, 145, 146, 157, 160, 171, 173, 175, 179, 180, 191, 192, 193, 196, 197,
                        198, 201, 203, 205, 207, 215, 216, 218, 219, 223, 226, 252, 253, 258, 260],

                #Bronx
                4 : [3, 18, 20, 31, 32, 46, 47, 51, 58, 59, 60, 69, 78, 81, 94, 119, 126, 136, 147, 159, 167, 168, 169, 174, 182, 183, 184,
                        185, 199, 200, 208, 212, 213, 220, 235, 240, 241, 242, 247, 248, 250, 254, 259],
                #State Island
                5 : [5, 6, 23, 44, 84, 99, 109, 110, 115, 118, 156, 172, 176, 187, 204, 206, 214, 221, 245, 251],

                #EWR
               6 : [1]}

    for borough, ids in zones.items():
        if location_id in ids:
            return borough
    return 7

meses = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
df_meses = []

for mes in meses:
    df_mes = pd.read_parquet(f'yellow_tripdata_2022-{mes}.parquet')
    df_mes = df_mes.sample(n=500, random_state=42).dropna()
    df_mes["PickupZone"] = df_mes["PULocationID"].apply(get_borough)
    df_mes['tpep_pickup_datetime'] = df_mes['tpep_pickup_datetime'].dt.hour
    df_meses.append(df_mes)

df = pd.concat(df_meses)
X = df[["tpep_pickup_datetime", 'trip_distance', 'PickupZone']]
y = df['total_amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Utiliza ElasticNet en lugar de RandomForestRegressor
model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Guarda el modelo entrenado para su uso en Streamlit
joblib.dump(model, 'ModeloEntrenado.pkl')

Este codigo se llama Stream.py y es el que estructura el Streamlit, se puede ver que se pide la Hora, las millas y la zona de donde se toma.

Omito los requirements.txt

In [ ]:
import streamlit as st
import pandas as pd
import joblib
import requests
from io import BytesIO
import sklearn


# Función para cargar el modelo desde GitHub
def load_model():
    # URL raw del archivo .pkl en GitHub
    model_url = 'https://github.com/darksider10/PF/raw/main/ML/ModeloEntrenado.pkl'
    
    # Descargar el archivo .pkl desde la URL
    response = requests.get(model_url)
    
    # Guardar el archivo .pkl localmente
    with open('ModeloEntrenado.pkl', 'wb') as f:
        f.write(response.content)
    
    # Cargar el modelo desde el archivo descargado
    model = joblib.load('ModeloEntrenado.pkl')
    return model

model = load_model()

# Creando la interfaz de usuario
st.title('Predicción del Total Amount para viajes de Taxi')

tpep_pickup_datetime= st.number_input('Hora', min_value=0, max_value=23)
trip_distance = st.number_input('Distancia del viaje (millas)', min_value=0.0, format="%.2f")
pickup_zone = st.selectbox('Zona de recogida', options=[('6','EWR'),('5', 'Staten Island'),('3', 'Queens'),('4', 'Bronx'), ('-1', 'Desconocido'),('2', 'Brooklyn'),('1', 'Manhattan')], format_func=lambda x: x[1])

# Botón para realizar la predicción
if st.button('Predecir Total Amount'):
    X_new = pd.DataFrame([[tpep_pickup_datetime, trip_distance, int(pickup_zone[0])]], columns=['tpep_pickup_datetime', 'trip_distance', 'PickupZone'])
    y_pred_new = model.predict(X_new)
    st.success(f'La predicción del total_amount es: ${y_pred_new[0]:.2f}')